In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%matplotlib inline

## **TASK 1. 데이터 출처 / 데이터 불러오기**
-  [kaggle_Health Insurance Cross Sell Prediction 🏠 🏥](https://www.kaggle.com/datasets/anmolkumar/health-insurance-cross-sell-prediction/data)
- 데이터 선정 이유: 보험 분야에서의 머신러닝 활용을 직접 해보고자!

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [3]:
train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [4]:
test.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,381110,Male,25,1,11.0,1,< 1 Year,No,35786.0,152.0,53
1,381111,Male,40,1,28.0,0,1-2 Year,Yes,33762.0,7.0,111
2,381112,Male,47,1,28.0,0,1-2 Year,Yes,40050.0,124.0,199
3,381113,Male,24,1,27.0,1,< 1 Year,Yes,37356.0,152.0,187
4,381114,Male,27,1,28.0,1,< 1 Year,No,59097.0,152.0,297


## **TASK 2. 기초 통계량 확인**

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   Gender                381109 non-null  object 
 2   Age                   381109 non-null  int64  
 3   Driving_License       381109 non-null  int64  
 4   Region_Code           381109 non-null  float64
 5   Previously_Insured    381109 non-null  int64  
 6   Vehicle_Age           381109 non-null  object 
 7   Vehicle_Damage        381109 non-null  object 
 8   Annual_Premium        381109 non-null  float64
 9   Policy_Sales_Channel  381109 non-null  float64
 10  Vintage               381109 non-null  int64  
 11  Response              381109 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 34.9+ MB


In [8]:
int_var = train[['id', 'Age', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage', 'Response']]
obj_var = train[['Gender', 'Vehicle_Age', 'Vehicle_Damage']]

### 수치형 변수(int, float)
- id: 고객 식별을 위한 변수 -> 통계 무의미
- Driving_License, Previously_Insured, Response(Y변수) : 범주형 변수(1/0)
- Region_Code, Policy_Sales_Channel: float이지만 사실상 범주형 변수
- **Annual_Premium, Vintage, Age** → 연속형 변수
  - Age: 20대 ~ 80대까지 존재 / 2-30대가 많은 듯?
  - Annual_Premium: 2630 ~ 540165 / 평균에 비해 최댓값이 커서 이상값 확인해보기 !
  - Vintage: 10~299

In [9]:
int_var.describe()

,id,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Response
count,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000
mean,190555.000000,38.822584,0.997869,26.388807,0.458210,30564.389581,112.034295,154.347397,0.122563
std,110016.836208,15.511611,0.046110,13.229888,0.498251,17213.155057,54.203995,83.671304,0.327936
min,1.000000,20.000000,0.000000,0.000000,0.000000,2630.000000,1.000000,10.000000,0.000000
25%,95278.000000,25.000000,1.000000,15.000000,0.000000,24405.000000,29.000000,82.000000,0.000000
50%,190555.000000,36.000000,1.000000,28.000000,0.000000,31669.000000,133.000000,154.000000,0.000000
75%,285832.000000,49.000000,1.000000,35.000000,1.000000,39400.000000,152.000000,227.000000,0.000000
max,381109.000000,85.000000,1.000000,52.000000,1.000000,540165.000000,163.000000,299.000000,1.000000


### 범주형 변수(object)
- gender: 남/여 → 남자 비율이 더 높음
- Vehicle_Age: 1-2년 정도된 자동차 소유자가 많음 
- Vehicle_Damage: 차량 손상여부 → 비슷함 
→ 활용전, 인코딩 필요! 


In [21]:
for i in range(obj_var.shape[1]):
    category = np.unique(obj_var.iloc[:,i])
    print(f'{obj_var.columns[i]}: {category} ')

Gender: ['Female' 'Male'] 
Vehicle_Age: ['1-2 Year' '< 1 Year' '> 2 Years'] 
Vehicle_Damage: ['No' 'Yes'] 


In [10]:
obj_var.describe()

,Gender,Vehicle_Age,Vehicle_Damage
count,381109,381109,381109
unique,2,3,2
top,Male,1-2 Year,Yes
freq,206089,200316,192413


### 변수 간단요약
<img src="https://github.com/leehj2586/Epoch/blob/main/Task/img/변수요약.png?raw=true" width="500">


## **TASK 3. 분석 목적/인사이트 설명**
- 목적: 건강 보험 가입 고객이 차량 보험에 가입할 가능성(Response=1)을 예측
    - 고객특성(연령, 운전면허 보유 여부, 차량 보유여부)와 차량특성(차량연식, 손상경험)을 이용하여 차량 보험 가능성 예측
- 문제 유형: 이진 분류(Binary Classification) 문제
- 얻을 수 있는 인사이트   
    1) 고객 특성과 보험 가입 가능성 간의 관계
        - 연령(Age)과 가입 의향(Response)의 관계
        - 운전면허 보유 여부(Driving_License)
        - 기존 차량 보험 여부(Previously_Insured)
    2) 차량 관련 정보와 가입 의향 분석
        - 차량 연식(Vehicle_Age)과 가입률
        - 사고 경험(Vehicle_Damage)과 가입 의향

### +) 여력이 된다면 해보고 싶은 것...
- 아이오와 주의 재산 및 상해 보험(Property Casualty Insurance) 보험료와 손실 데이터
    - 보험사가 지급해야 할 손실(Losses Incurred)을 예측하여 손해율 관리 및 보험료 최적화
    - 하지만 지금은 시계열도 보험도 하나도 모르기 때문에 도저히 단기간에 할 수가 없다는 점..